In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the Excel file
data = pd.read_csv("SBI.csv")

# Assuming the ' Adj Close' column contains the adj. closing prices
data['Returns'] = data['Adj Close'].pct_change().dropna()

# List of confidence levels
confidence_levels = [0.90, 0.95, 0.99]

# Function to calculate Historical VaR and ES
def historical_var_es(returns, confidence_level):
    alpha = 1 - confidence_level
    sorted_returns = np.sort(returns)
    var_index = int(alpha * len(sorted_returns))
    
    # VaR is the alpha-th percentile
    VaR = sorted_returns[var_index]
    
    # ES is the average of the returns worse than the VaR
    ES = np.mean(sorted_returns[:var_index])
    
    return VaR, ES

# Function to calculate Monte Carlo VaR and ES
def monte_carlo_var_es(returns, confidence_level, num_simulations=10000):
    alpha = 1 - confidence_level
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    
    simulated_returns = np.random.normal(mean_return, std_return, num_simulations)
    sorted_simulated_returns = np.sort(simulated_returns)
    var_index = int(alpha * len(sorted_simulated_returns))
    
    # VaR from the Monte Carlo simulated returns
    VaR = sorted_simulated_returns[var_index]
    
    # ES from the Monte Carlo simulated returns
    ES = np.mean(sorted_simulated_returns[:var_index])
    
    return VaR, ES

# Extract the returns for calculation
returns = data['Returns'].dropna().values

# Calculate VaR and ES for each confidence level
for confidence_level in confidence_levels:
    # Historical Method
    historical_VaR, historical_ES = historical_var_es(returns, confidence_level)
    
    # Monte Carlo Simulation
    mc_VaR, mc_ES = monte_carlo_var_es(returns, confidence_level)
    
    print(f"Confidence Level: {confidence_level*100}%")
    print(f"Historical VaR: {historical_VaR*100:.2f}%, Historical ES: {historical_ES*100:.2f}%")
    print(f"Monte Carlo VaR: {mc_VaR*100:.2f}%, Monte Carlo ES: {mc_ES*100:.2f}%")
    print("-" * 50)


Confidence Level: 90.0%
Historical VaR: -2.55%, Historical ES: -4.05%
Monte Carlo VaR: -2.97%, Monte Carlo ES: -4.02%
--------------------------------------------------
Confidence Level: 95.0%
Historical VaR: -3.56%, Historical ES: -5.10%
Monte Carlo VaR: -3.80%, Monte Carlo ES: -4.80%
--------------------------------------------------
Confidence Level: 99.0%
Historical VaR: -6.08%, Historical ES: -7.90%
Monte Carlo VaR: -5.32%, Monte Carlo ES: -6.15%
--------------------------------------------------


In [30]:
current_market_price = 782  # Set your current market price

# Calculate VaR and ES for each confidence level
for confidence_level in confidence_levels:
    print(f"Confidence Level: {confidence_level * 100}%")
    
    # Historical Method
    historical_VaR, historical_ES = historical_var_es(returns, confidence_level)
    historical_var_amount = historical_VaR * current_market_price
    historical_es_amount = historical_ES * current_market_price
    
    print(f"Historical VaR in Amount: ₹{historical_var_amount:.2f}, Historical ES in Amount: ₹{historical_es_amount:.2f}")
    
    # Monte Carlo Simulation
    mc_VaR, mc_ES = monte_carlo_var_es(returns, confidence_level)
    mc_var_amount = mc_VaR * current_market_price
    mc_es_amount = mc_ES * current_market_price
    
    print(f"Monte Carlo VaR in Amount: ₹{mc_var_amount:.2f}, Monte Carlo ES in Amount: ₹{mc_es_amount:.2f}")
    print("-" * 50)

Confidence Level: 90.0%
Historical VaR in Amount: ₹-19.90, Historical ES in Amount: ₹-31.67
Monte Carlo VaR in Amount: ₹-23.44, Monte Carlo ES in Amount: ₹-32.50
--------------------------------------------------
Confidence Level: 95.0%
Historical VaR in Amount: ₹-27.84, Historical ES in Amount: ₹-39.88
Monte Carlo VaR in Amount: ₹-29.42, Monte Carlo ES in Amount: ₹-37.37
--------------------------------------------------
Confidence Level: 99.0%
Historical VaR in Amount: ₹-47.55, Historical ES in Amount: ₹-61.79
Monte Carlo VaR in Amount: ₹-41.51, Monte Carlo ES in Amount: ₹-49.05
--------------------------------------------------


In [57]:
# Prepare an empty list to store results
historical_results = []
mc_results = []

# Calculate VaR and ES for each confidence level
for confidence_level in confidence_levels:
    # Historical Method
    historical_VaR, historical_ES = historical_var_es(returns, confidence_level)
    historical_var_amount = historical_VaR * current_market_price
    historical_es_amount = historical_ES * current_market_price
    
    # Append the results for Historical method
    historical_results.append({
        "Confidence Level": f"{confidence_level * 100}%",
        "VaR (%)": f"{historical_VaR * 100:.2f}%",
        "ES (%)": f"{historical_ES * 100:.2f}%",
        "VaR (₹)": f"₹{historical_var_amount:.2f}",
        "ES (₹)": f"₹{historical_es_amount:.2f}"
    })
    
    # Monte Carlo Simulation
    mc_VaR, mc_ES = monte_carlo_var_es(returns, confidence_level)
    mc_var_amount = mc_VaR * current_market_price
    mc_es_amount = mc_ES * current_market_price
    
    # Append the results for Monte Carlo method
    mc_results.append({
        "Confidence Level": f"{confidence_level * 100}%",
        "VaR (%)": f"{mc_VaR * 100:.2f}%",
        "ES (%)": f"{mc_ES * 100:.2f}%",
        "VaR (₹)": f"₹{mc_var_amount:.2f}",
        "ES (₹)": f"₹{mc_es_amount:.2f}"
    })

# Convert the results into DataFrames for tabular display
df_historical = pd.DataFrame(historical_results)
df_mc = pd.DataFrame(mc_results)

# Display the separate tables
print("The Current Market Price of SBI :",current_market_price)
print("-"*60)
print("Historical Method Results:")
print(df_historical)
print("-"*60)
print("\nMonte Carlo Method Results:")
print(df_mc)
print("-"*60)

The Current Market Price of SBI : 782
------------------------------------------------------------
Historical Method Results:
  Confidence Level VaR (%)  ES (%)  VaR (₹)   ES (₹)
0            90.0%  -2.55%  -4.05%  ₹-19.90  ₹-31.67
1            95.0%  -3.56%  -5.10%  ₹-27.84  ₹-39.88
2            99.0%  -6.08%  -7.90%  ₹-47.55  ₹-61.79
------------------------------------------------------------

Monte Carlo Method Results:
  Confidence Level VaR (%)  ES (%)  VaR (₹)   ES (₹)
0            90.0%  -2.92%  -4.08%  ₹-22.86  ₹-31.88
1            95.0%  -3.82%  -4.77%  ₹-29.89  ₹-37.34
2            99.0%  -5.30%  -6.16%  ₹-41.48  ₹-48.20
------------------------------------------------------------
